In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

In [2]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])

    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)

def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [3]:
data = pd.read_csv("C:\\Users\\tanbe\\Desktop\\transaction_data.csv")

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [4]:
item_features = pd.read_csv("C:\\Users\\tanbe\\Desktop\\product.csv")
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [5]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [7]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [8]:
data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [9]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

c:\users\tanbe\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [11]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице !

# ALS

In [12]:
%%time

model = AlternatingLeastSquares(factors=64, 
                                regularization=0.05,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=10,
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 3.94 s


In [13]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [14]:
%%time

params = {'factors':[ 40, 80, 120, 140], 
          'regularization':[ 0.001, 0.05]}

for factor in params['factors']:
    for reg in params['regularization']:
        model = AlternatingLeastSquares(factors=factor, 
                                regularization=reg,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4,
                                random_state=42)

        model.fit(csr_matrix(user_item_matrix).T,  # На вход item-user matrix
                  show_progress=False)


        result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        sc = result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()
        print(f'Factor:{factor:3},  Regularization:{reg:6}: p_at_k:{sc:7.5f}')

Factor: 40,  Regularization: 0.001: p_at_k:0.16303
Factor: 40,  Regularization:  0.05: p_at_k:0.16585
Factor: 80,  Regularization: 0.001: p_at_k:0.18021
Factor: 80,  Regularization:  0.05: p_at_k:0.18192
Factor:120,  Regularization: 0.001: p_at_k:0.18272
Factor:120,  Regularization:  0.05: p_at_k:0.18644
Factor:140,  Regularization: 0.001: p_at_k:0.19669
Factor:140,  Regularization:  0.05: p_at_k:0.19618
Wall time: 5min 1s


In [15]:
params = {'factors':[ 160, 180, 200], 
          'regularization':[ 0.001, 0.05]}

for factor in params['factors']:
    for reg in params['regularization']:
        model = AlternatingLeastSquares(factors=factor, 
                                regularization=reg,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4,
                                random_state=42)

        model.fit(csr_matrix(user_item_matrix).T,  # На вход item-user matrix
                  show_progress=False)


        result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        sc = result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()
        print(f'Factor:{factor:3},  Regularization:{reg:6}: p_at_k:{sc:7.5f}')

Factor:160,  Regularization: 0.001: p_at_k:0.19247
Factor:160,  Regularization:  0.05: p_at_k:0.19257
Factor:180,  Regularization: 0.001: p_at_k:0.19307
Factor:180,  Regularization:  0.05: p_at_k:0.19468
Factor:200,  Regularization: 0.001: p_at_k:0.19669
Factor:200,  Regularization:  0.05: p_at_k:0.19578


Лучше всего себя показала модель с параметрами: Factor:140, Regularization: 0.001.

# Ответы на вопросы

**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

- А точно нужно сортировать по вероятности?

Сортировать по вероятности рекомендуется, так как рассылаем только 5% и 20% клиентов, однако можно и по потенциальной выручке с товара.

- Какую метрику использовать?

Я считаю, что в этом случае precision - лучший вариант, так как она учитывает стоимость каждого рекомендованного товара.

- Сколько раз в неделю отправляем рассылку?

Думаю, что два раза в неделю будет достаточно, один день - будний (к примеру, среда) и выходной (суббота). 

- В какое время отправляем рассылку?

Здесь важно учитывать время, когда человек будет не занят и точно проанализирует информацию в рассылке - то есть это должно быть время после рабочего дня, примерно в 19-20 часов по будням, а по выходным в 10-12 часов (перед походом в магазин).

- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?

Можно рекомендовать товары из похожих групп, товары, которые обычно покупают вместе с уже купленными товарами, а также товары, которые приобретали покупатели, чьи интересы совпадают с пользователем.

- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?

Можно предлагать пользователю как разные товары (которые обычно покупаются вместе с основным), так можно и присылать одинаковые товары, но разных брендов (например, чтобы пользователь мог сравнить цены или характеристики).

Чтобы увидеть, разные ли товары, то можно проанализировать, какие из них никогда или очень редко покупаются вместе - это может показывать, что товары одинаковые. Чем чаще товары приобретаются вместе - тем больше их разница.